# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re
import os
import pickle
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_multilabel_classification
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.svm import SVC
from copy import deepcopy
import nltk
nltk.download('stopwords')
nltk.download(['punkt', 'wordnet'])
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
print(os.getcwd())

/home/workspace


In [40]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('DisasterResponse', engine)
X = df['message']
y = df[df.columns[5:]]
#y = df.drop(['message', 'genre', 'id', 'original'], axis=1)

In [30]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26216 entries, 0 to 26215
Data columns (total 35 columns):
request                   26216 non-null int64
offer                     26216 non-null int64
aid_related               26216 non-null int64
medical_help              26216 non-null int64
medical_products          26216 non-null int64
search_and_rescue         26216 non-null int64
security                  26216 non-null int64
military                  26216 non-null int64
child_alone               26216 non-null int64
water                     26216 non-null int64
food                      26216 non-null int64
shelter                   26216 non-null int64
clothing                  26216 non-null int64
money                     26216 non-null int64
missing_people            26216 non-null int64
refugees                  26216 non-null int64
death                     26216 non-null int64
other_aid                 26216 non-null int64
infrastructure_related    26216 non-null int6

### 2. Write a tokenization function to process your text data

In [41]:
def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [42]:
#using MultiOutputClassifier to build an ML pipeline
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [44]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((5243,), (20973,), (5243, 35), (20973, 35))

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [50]:
y_pred = pipeline.predict(X_test)

print(classification_report(y_test, y_pred, target_names = y.columns))

                        precision    recall  f1-score   support

               request       0.81      0.37      0.51      1111
                 offer       0.00      0.00      0.00        33
           aid_related       0.74      0.51      0.61      2670
          medical_help       0.61      0.07      0.12       535
      medical_products       0.79      0.08      0.14       344
     search_and_rescue       0.60      0.04      0.07       159
              security       0.20      0.01      0.02       116
              military       0.50      0.06      0.10       200
           child_alone       0.00      0.00      0.00         0
                 water       0.83      0.28      0.42       418
                  food       0.82      0.27      0.40       745
               shelter       0.77      0.16      0.27       581
              clothing       0.56      0.05      0.09        98
                 money       0.78      0.05      0.10       133
        missing_people       0.33      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [49]:
# def display_results(cv, y_test, y_pred):
#     labels = np.unique(y_pred)
#     confusion_mat = confusion_matrix(y_test, y_pred, labels=labels)
#     accuracy = (y_pred == y_test).mean()
#     cr_y1 = classification_report(y_test[:,0],y_pred[:,0])
#      print("Labels:", labels)
#     print("Confusion Matrix:\n", confusion_mat)
#     print("Accuracy:", accuracy)
#     #print("\nBest Parameters:", cv.best_params_)
#     print("f1 score:", cr_y1)


### 6. Improve your model
Use grid search to find better parameters. 

In [55]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f0f425880d0>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [57]:
parameters = {'clf__estimator__max_depth': [10, 50, None],
              'clf__estimator__min_samples_leaf':[2, 5, 10],
             'clf__estimator__min_samples_split': [2, 4],}

cv = GridSearchCV(pipeline, parameters)
cv

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__max_depth': [10, 50, None], 'clf__estimator__min_samples_leaf': [2, 5, 10], 'clf__estimator__min_samples_split': [2, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [60]:
cv = GridSearchCV(pipeline, param_grid = parameters,verbose = 10)
np.random.seed(42)
y_pred2 = cv.fit(X_train, y_train)

Fitting 3 folds for each of 18 candidates, totalling 54 fits
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=2, clf__estimator__min_samples_split=2 
[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=2, clf__estimator__min_samples_split=2, score=0.4189807750991761, total=  13.4s
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=2, clf__estimator__min_samples_split=2 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   20.9s remaining:    0.0s


[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=2, clf__estimator__min_samples_split=2, score=0.41684467500762895, total=  13.3s
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=2, clf__estimator__min_samples_split=2 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   41.9s remaining:    0.0s


[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=2, clf__estimator__min_samples_split=2, score=0.4284406469331706, total=  13.4s
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=2, clf__estimator__min_samples_split=4 


...

[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:  2.9min remaining:    0.0s


[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=5, clf__estimator__min_samples_split=2, score=0.4250839182178822, total=  14.4s
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=5, clf__estimator__min_samples_split=4 


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  3.2min remaining:    0.0s


[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=5, clf__estimator__min_samples_split=4, score=0.42340555386023804, total=  14.6s
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=5, clf__estimator__min_samples_split=4 
[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=5, clf__estimator__min_samples_split=4, score=0.41959108941104667, total=  13.7s
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=5, clf__estimator__min_samples_split=4 
[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=5, clf__estimator__min_samples_split=4, score=0.4270674397314617, total=  13.2s
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=10, clf__estimator__min_samples_split=2 
[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=10, clf__estimator__min_samples_split=2, score=0.42340555386023804, total=  13.4s
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=10, clf

[Parallel(n_jobs=1)]: Done  54 out of  54 | elapsed: 26.1min finished


In [62]:
prediction2 = y_pred2.predict(X_test)

print(classification_report(y_test, prediction2 , target_names = y.columns))

                        precision    recall  f1-score   support

               request       1.00      0.00      0.00      1111
                 offer       0.00      0.00      0.00        33
           aid_related       0.88      0.12      0.22      2670
          medical_help       0.00      0.00      0.00       535
      medical_products       0.00      0.00      0.00       344
     search_and_rescue       0.00      0.00      0.00       159
              security       0.00      0.00      0.00       116
              military       0.00      0.00      0.00       200
           child_alone       0.00      0.00      0.00         0
                 water       0.00      0.00      0.00       418
                  food       0.00      0.00      0.00       745
               shelter       0.00      0.00      0.00       581
              clothing       0.00      0.00      0.00        98
                 money       0.00      0.00      0.00       133
        missing_people       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [63]:
pipeline2 = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
                ('tfidf_transformer', TfidfTransformer())
            ])),
       ])),

        ('classifier', MultiOutputClassifier(AdaBoostClassifier()))
    ])

In [64]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

pipeline2.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('count_vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

In [65]:
y_pred3 = pipeline2.predict(X_test)

print(classification_report(y_test, y_pred3, target_names = y.columns))

                        precision    recall  f1-score   support

               request       0.77      0.55      0.64      1111
                 offer       0.00      0.00      0.00        33
           aid_related       0.75      0.59      0.66      2670
          medical_help       0.57      0.25      0.35       535
      medical_products       0.65      0.35      0.45       344
     search_and_rescue       0.51      0.15      0.23       159
              security       0.14      0.03      0.04       116
              military       0.56      0.27      0.36       200
           child_alone       0.00      0.00      0.00         0
                 water       0.72      0.61      0.66       418
                  food       0.81      0.68      0.74       745
               shelter       0.77      0.58      0.66       581
              clothing       0.68      0.43      0.53        98
                 money       0.46      0.28      0.35       133
        missing_people       0.73      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 9. Export your model as a pickle file

In [3]:
# Pickle best model
pickle.dump(y_pred2, open('disaster_model.sav', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

b'\x80\x03X\x0e\x00\x00\x00classifier.pklq\x00.'